In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv, find_dotenv
import os

In [5]:
load_dotenv(find_dotenv())
filepath_parquet=os.getenv("FILEPATH_PARQUET")
filepath_csv_products=os.getenv("FILEPATH_CSV_PRODUCTS")
filepath_csv_extras=os.getenv("FILEPATH_CSV_EXTRAS")
base_url=os.getenv("BASE_URL")

def html_to_text(text):
    soup = BeautifulSoup(text)
    return soup.get_text()

def append_products(text):
    return base_url+"products/"+text
def append_extras(text):
    return base_url+text


def pandas_with_text(filepath):
    df=pd.read_csv(filepath)
    df_filtered=df[~df["Body (HTML)"].isnull()][["Handle","Body (HTML)","SEO Description","Variant Price"]]
    df_filtered["text"]=df_filtered["Body (HTML)"].apply(html_to_text)
    df_filtered["source"]=df_filtered["Handle"].apply(append_source)
    df_filtered=df_filtered.rename(columns={"Handle":"id"})
    return df_filtered[["id","text","source"]]






In [12]:
df_products=pandas_with_text(filepath_csv_products)

In [13]:
df_extras=pd.read_csv(filepath_csv_extras)
df_extras["source"]=df_extras["id"].apply(append_extras)

In [14]:
df_all=pd.concat([df_products,df_extras]).reset_index(drop=True)
df_all.to_parquet(filepath_parquet)